In [1]:
import sys

from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
import pyspark.sql.functions as F

import os
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'

import findspark
findspark.init()
findspark.find()

'/usr/local/lib/python3.8/dist-packages/pyspark'

In [2]:
import datetime
import pyspark.sql.functions as F
from pyspark.sql.window import Window
 
def input_event_paths(date, depth):
    dt = datetime.datetime.strptime(date, '%Y-%m-%d')
    return [f"/user/username/data/events/date={(dt-datetime.timedelta(days=x)).strftime('%Y-%m-%d')}" for x in range(depth)]

def get_spark():
    conf = SparkConf().setAppName(f"EventsPartitioningJob_i028159")
    sc = SparkContext(conf=conf)
    return SQLContext(sc)

def main():
    date = '2022-05-04'
    base_input_path = '/user/master/data/events'
    base_output_path = 'hdfs://rc1a-dataproc-m-dg5lgqqm7jju58f9.mdb.yandexcloud.net/user/inthemidde/data/events'
    depth = 5
    
    sql = get_spark()
    print(sql)
    
    for i in range(0,5,1): #range(start, stop[, step]):
        dt = datetime.datetime.strptime(date, '%Y-%m-%d')
        lv_date = (dt-datetime.timedelta(days=i)).strftime('%Y-%m-%d')
        print(lv_date)
        events = sql.read.json(f"{base_input_path}/date={lv_date}")
        writer = partition_writer(events)
        writer.save(f'{base_output_path}/date={lv_date}')
        rr = tag_tops(dt-datetime.timedelta(days=i), depth, sql)

In [4]:
def partition_writer(events):
    return events \
        .write \
        .mode('overwrite') \
        .partitionBy('event_type') \
        .format('parquet')

In [6]:
def tag_tops(date, depth, spark):
    
		message_paths = input_event_paths(date, depth)
		result = spark.read\
    		.option("basePath", "/user/username/data/events")\
    		.parquet(*message_paths)\
    		.where("event_type = 'message'")\
		.where("event.message_channel_to is not null")\
		.select(F.col("event.message_id").alias("message_id"),
		        F.col("event.message_from").alias("user_id"),
		        F.explode(F.col("event.tags")).alias("tag"))\
		.groupBy("user_id", "tag")\
		.agg(F.count("*").alias("tag_count"))\
		.withColumn("rank", F.row_number().over(Window.partitionBy("user_id")\
        	.orderBy(F.desc("tag_count"), F.desc("tag"))))\
		.where("rank <= 3")\
		.groupBy("user_id")\
		.pivot("rank", [1, 2, 3])\
		.agg(F.first("tag"))\
		.withColumnRenamed("1", "tag_top_1")\
		.withColumnRenamed("2", "tag_top_2")\
		.withColumnRenamed("3", "tag_top_3")
 
		return result

In [7]:
main()

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=EventsPartitioningJob_i028159, master=local[*]) created by __init__ at /tmp/ipykernel_3927/3392455117.py:11 